<h1>Проект по SQL</h1>

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато появилось больше времени на книги. Стартапы заметили это и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис подписки на чтение книг.

Задача проекта - проанализировать базу данных.

Цели исследования:

- Подсчитать, сколько книг было издано после 1 января 2000 года.
- Для каждой книги подсчитать количество рецензий и средний рейтинг.
- Определить издательство, выпустившее наибольшее количество книг объемом более 50 страниц.
- Найти автора с самым высоким средним рейтингом для книги, учитывая только книги с 50 и более рейтингами.
- Подсчитайть среднее количество рецензий от пользователей, поставивших более 48 оценок.

In [ ]:
import pandas as pd
from sqlalchemy import text, create_engine

In [ ]:
db_config = {'user': 'praktikum_student',
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432,
'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [ ]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#### Исследование таблиц. Вывод первых строк и подсчет количества строк в каждой таблице.

Вывод первых строк и подсчет количества строк в таблице books.

In [ ]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
query = '''SELECT COUNT(*) AS book_count FROM books'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_count
0,1000


Вывод первых строк и подсчет количества строк в таблице authors.

In [ ]:
query = '''SELECT * FROM authors LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
query = '''SELECT COUNT(*) AS author_count FROM authors;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_count
0,636


Вывод первых строк и подсчет количества строк в таблице publishers.

In [ ]:
query = '''SELECT * FROM publishers LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
query = '''SELECT COUNT(*) AS publisher_count FROM publishers;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_count
0,340


Вывод первых строк и подсчет количества строк в таблице ratings.

In [ ]:
query = '''SELECT * FROM ratings LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
query = '''SELECT COUNT(*) AS rating_count FROM ratings;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_count
0,6456


Вывод первых строк и подсчет количества строк в таблице reviews

In [ ]:
query = '''SELECT * FROM reviews LIMIT 5;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [ ]:
query = '''SELECT COUNT(*)AS review_count FROM reviews;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_count
0,2793


#### Посчитаем, сколько книг вышло после 1 января 2000 года.

In [ ]:
query = '''SELECT COUNT(*) AS books_published_after_2000
FROM books
WHERE publication_date > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,books_published_after_2000
0,819


После 1 января 2000 года вышло 819 книг.

#### Теперь посчитаем для каждой книги количество обзоров и среднюю оценку.

In [ ]:
query = '''
    SELECT
        b.book_id,
        b.title,
        COUNT(DISTINCT r.review_id) AS review_count,
        AVG(rt.rating) AS average_rating
    FROM
        books b
    LEFT JOIN
        reviews r ON b.book_id = r.book_id
    LEFT JOIN
        ratings rt ON b.book_id = rt.book_id
    GROUP BY
        b.book_id, b.title
    ORDER BY
        review_count DESC;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Максимальное число ревью у `Сумерок, Воды слонам, Стеклянный замок и Гарри Поттера`.

Посчитаем сумму значений review_count из результата запроса:

In [ ]:
query = '''
    SELECT
        SUM(review_count) AS total_review_count
    FROM (
        SELECT
            COUNT(DISTINCT r.review_id) AS review_count
        FROM
            books b
        LEFT JOIN
            reviews r ON b.book_id = r.book_id
        LEFT JOIN
            ratings rt ON b.book_id = rt.book_id
        GROUP BY
            b.book_id, b.title
    ) AS subq;
'''

total_review_count_after_query = pd.read_sql(query, engine)
print("Общее количество обзоров:", total_review_count_after_query)

Общее количество обзоров:    total_review_count
0              2793.0


#### Определим  издательство, которое выпустило наибольшее количество книг толще 50 страниц.

In [ ]:
query = '''SELECT
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS books_published
FROM
    books b
JOIN
    publishers p ON b.publisher_id = p.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher_id,
    p.publisher
ORDER BY
    COUNT(b.book_id) DESC
LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,books_published
0,212,Penguin Books,42


Издательство Penguin Books выпустило наибольшее количество книг толще 50 страниц.

#### Найдем автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [ ]:
query = '''SELECT
    b.author_id,
    a.author,
    AVG(r.rating) AS average_rating
FROM
    (SELECT
        book_id,
        author_id
    FROM
        books
    WHERE
        book_id IN (
            SELECT
                book_id
            FROM
                ratings
            GROUP BY
                book_id
            HAVING
                COUNT(rating_id) >= 50
        )) AS b
JOIN
    authors a ON b.author_id = a.author_id
JOIN
    ratings r ON b.book_id = r.book_id
GROUP BY
    b.author_id,
    a.author
ORDER BY
    AVG(r.rating) DESC
LIMIT 1;'''

con = engine.connect()
pd.io.sql.read_sql(sql=text(query), con=con)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг: J.K. Rowling и Mary GrandPré

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [ ]:
query = '''SELECT
    AVG(sub.review_count) AS average_review_count
FROM (
    SELECT
        username,
        COUNT(review_id) AS review_count
    FROM
        reviews
    WHERE
        username IN (
            SELECT
                username
            FROM
                ratings
            GROUP BY
                username
            HAVING
                COUNT(rating_id) > 48
        )
    GROUP BY
        username
) AS sub;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,average_review_count
0,24.0


Среднее количество обзоров от пользователей, поставивших больше 48 оценок: 24 обзора.

#### Заключение:

1. **Популярные книги:**.
   - Наибольшее количество рецензий набрали книги `Twilight (Twilight #1), Water for Elephants, The Glass Castle, Harry Potter and the Prisoner of Azkaban, The Curious Incident of the Dog in the Night-Time.`  Именно по этим книгам были сняты экранизации, что может быть связано с их широкой популярностью и говорит об их высокой популярности среди читателей.
   
2. **Penguin Books:** Издатель.
   - Издательство Penguin Books опубликовало наибольшее количество книг объемом более 50 страниц. Это говорит о широком каталоге издательства и, возможно, о его долгой истории на рынке книгоиздания.
   
3. **Автор с самым высоким средним рейтингом:**.
   -  J.K. Rowling и Mary GrandPré имеет самый высокий средний рейтинг книг среди авторов в наборе данных. Это может свидетельствовать о высоком качестве произведений и предпочтениях среди читателей.
    
4. **Среднее количество пользовательских рецензий:**.
   - Если рассматривать пользователей, оставивших более 48 оценок, то их среднее количество рецензий составляет 24. Это может свидетельствовать о том, что эти пользователи активно участвуют в жизни сообщества, обсуждают прочитанные книги и предпочитают делиться своим мнением.  
   
Основываясь на этих данных, мы можем сформулировать ценностное предложение нового продукта следующим образом:

"Наш новый сервис чтения книг по подписке предлагает доступ к широкому каталогу книг от различных издательств, включая лучшие произведения авторов, которые высоко ценятся читателями, таких как J.K. Rowling. Мы создаем сообщество любителей чтения, где пользователи могут обмениваться рецензиями и рекомендациями. Мы также поддерживаем такие классические произведения, как "Алиса в Стране чудес" и "Вырастить дерево в Бруклине", которые всегда остаются в центре внимания читателей. Присоединяйтесь к нам, чтобы принять участие в захватывающем обсуждении книг с единомышленниками!"